In [1]:
import numpy as np
import cv2
import sys
import math

### 1, 2, 3. Function that gets the image file or capture it from camera

In [2]:
def get_image():
#   read from file or videocapture
    file = input("Write the file name path or press enter to capture an image: \n")
    if len(file) > 1:
        image_original = cv2.imread(file,1)
    else:
        cap = cv2.VideoCapture(0)
        retval,image_original = cap.read()
#   Conversion to 3 color channel
    image_original = to3channel(image_original)
#   Program should work for any size
    while image_original.shape[0] > 1200 or image_original.shape[1] > 750:
        image_original = cv2.resize(image_original,(int(image_original.shape[1]/2), 
                                                    int(image_original.shape[0]/2)))
    cv2.imshow ('image', image_original)
    return (image_original, file)

### 5

### a. Function that reloads the original image, pressed "i"

In [3]:
def reload_image(image):
    if len(image) > 1:
        image_original = cv2.imread(image)
    else:
        cap = cv2.VideoCapture(0)
        retval,image_original = cap.read()
    while image_original.shape[0] > 1200 or image_original.shape[1] > 750:
        image_original = cv2.resize(image_original,(int(image_original.shape[1]/2), 
                                                    int(image_original.shape[0]/2)))
    return image_original

### b. Function that save current image to file, pressed "w"

In [4]:
def savefile(image):
    cv2.imwrite("out.jpg", image)

### c. Function that converts to gray with cv2, pressed "g"

In [5]:
def togray(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return image

### d. Function that converts to gray with own implementation, pressed "G"

In [6]:
def tograyown(image):
    lumi=np.dot(image[...,:3], [0.299, 0.587, 0.114])
    return lumi.astype(np.uint8)

### e. Function that cycles through color channels, pressed "c"

In [7]:
def changecolor(image, count):
    if len(image.shape) == 3:
        if count == 1:
            image[:,:,1] = 0
            image[:,:,2] = 0
            count = 1
            print("blue channel showed")
        elif count == 2:
            image[:,:,0] = 0
            image[:,:,2] = 0
            count = 2
            print("green channel showed")
        else:
            image[:,:,0] = 0
            image[:,:,1] = 0
            count = 1
            print("red channel showed")
    else:
        print("Not possible to convert to b, g or r")
    return image

### f. Function that smooths the image using opencv, pressed "s"

In [8]:
def smooth(self):
    n = self
    image = reload_image(file)
    image = togray(image)
    if self != 0:
        kernel = np.ones((n,n), np.float32)/(n*n)
        image = cv2.filter2D(image, -1, kernel)
    cv2.imshow('image', image)

### g. Function that smooths the image using own implementation with finction "conv", pressed "S"

In [9]:
def smoothown(self):
    aux=0
    if self%2 !=1:
        aux=1
    n = self + aux
    image = reload_image(file)
    image = tograyown(image)
    if self != 0:
        kernel = np.ones((n,n), np.float32)/(n*n)
        image = conv(image, kernel)
    cv2.imshow('image', image)

### Convolution function

In [10]:
def conv(image, kernel):
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    pad = int(math.floor(kW/2))
    image = cv2.copyMakeBorder(image, pad, pad , pad , pad , cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float32")
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            roi = image[y - pad : y + pad +1, x - pad : x + pad +1]
            k = (roi * kernel).sum()
            output[y - pad, x - pad] = k
    return output.astype(np.uint8)

### h. Function that downsamples the image by 2 factor with no smoothing, pressed "d"

In [11]:
def downsample(image):
    image = cv2.resize(image, (int(image.shape[1]/2), int(image.shape[0]/2)))
    return image

### i. Function that downsamples the image by 2 factor with smoothing, pressed "D"

In [12]:
def downsample_smooth(image):
    image = cv2.pyrDown(image)
    return image

### j,k. Function that makes the convolution with a x derivative filter and normalizaed, pressed "x" or "y"

In [13]:
def convdevnorm(image,xory):
    aux=0
    if (xory==1):
        aux=0
    else:
        aux=1
    sobelx = cv2.Sobel(image,cv2.CV_64F,xory,aux,ksize=5)
    image = cv2.normalize(sobelx, image, alpha = 0, beta = 1,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    return image

### l. Function that shows magnitude of the gradient computed with x and y derivatives in range [0,255], pressed "m"

In [14]:
def gradient_magnitude(image):
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)
    gradient = cv2.magnitude(sobelx, sobely)
    image = cv2.normalize(gradient, image, alpha = 0, beta = 1,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    return image

### m. Funtion that plots the gradient vectors of the image, pressed "p"

In [15]:
def gradient_vectors(self):
    n = self
    image = reload_image(file)
    image = togray(image)
    grad = image
    if self != 0:
        sobelx = cv2.Sobel(grad,cv2.CV_64F,1,0,ksize=5)
        sobely = cv2.Sobel(grad,cv2.CV_64F,0,1,ksize=5)
        for x in range(0, grad.shape[0], n):
            for y in range(0, grad.shape[1], n):
                grad_angle = math.atan2(sobely[x, y], sobelx[x, y])
                grad_x = int(x + n * math.cos(grad_angle))
                grad_y = int(y + n * math.sin(grad_angle))
                cv2.arrowedLine(grad, (y, x), (grad_y, grad_x), (0, 0, 0))
    cv2.imshow('image', grad)
    image = grad

### n. Funtion that rotates the image, pressed "r"

In [16]:
def rotate(self):
    n = self
    image = reload_image(file)
    image = togray(image)
    rotated = image
    if self != 0:
        rot = cv2.getRotationMatrix2D((rotated.shape[1]/2, rotated.shape[0]/2), n, 1)
        rotated = cv2.warpAffine(rotated, rot,(rotated.shape[1], rotated.shape[0]))
    cv2.imshow('image', rotated)
    image = rotated

### o. Function that prints help about the use of the keyboard keys for the program , pressed "h"

In [17]:
def help():
    print("Press ‘i’ to reload the original image. \n")
    print("Press ‘w’ to save the current image into the file 'ouput.jpg' \n")
    print("Press ‘g’ to convert the image to grayscale using the OpenCV conversion function \n")
    print("Press ‘G’ to convert the image to grayscale using your implementation of conversion function. \n")
    print("Press ‘c’ to cycle through the color channels of the image showing a different channel every time the key is pressed. \n")
    print("Press ‘s’ to convert the image to grayscale and smooth it using the openCV function. Use a track bar to control the amount of smoothing. \n")
    print("Press ‘S’ to convert the image to grayscale and smooth it using your function which should perform convolution with a suitable filter. Use a track bar to control the amount of smoothing, \n")
    print("Press ‘d’ to downsample the image by a factor of 2 without smooting. \n")
    print("Press ‘D’ to downsample the image by a factor of 2 with smoothing. \n")
    print("Press ‘x’ to convert the image grayscale and perform convolution with an x derivative filter. Normalize the obtained values to the range [0,255]. \n")
    print("Press ‘y’ to convert the image to grayscale and perform convolution with a y derivative filter. Normalize the obtained values to the range [0,255]. \n")
    print("Press ‘m’ to show the magnitude of the gradient normalized to the range [0,255]. The gradient is computed base don the x and y derivatives of the image. \n")
    print("Press ‘p’ to convert the image to grayscale and plot the gradient vectors of the image every N pixel and let the plotted gradient vectors have a lenght of K. Use a track bar to control N. Plot the vectors as short line segments of length K. \n")
    print("Press ‘r’ to convert the image to grayscale and rotate it using an angle of teta degrees. Use a track bar to control the rotation angle. \n")
    print("Press ‘h’ to display a short description of the program, its command line arguments, and the keys it supports. \n")

### Converts image to RGB channelels

In [18]:
def to3channel(image):
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return image

### Main method

In [19]:
def main():
    global image
    global file
    image, file = get_image()
    print("Press h for help")
#   count for switching between RGB channels
    count = 0 
    while(True):
        key = cv2.waitKey()
        print (key)
        if key == ord('i'):
            image = reload_image(file)
            print("'i' key pressed: image reloaded")
        
        elif key == ord('w'):
            savefile(image)
            print("'w' key pressed: image saved into out.jpg file")
        
        elif key == ord('g'):
            image = reload_image(file)
            image = togray(image)
            print("'g' key pressed: convert to grey using openCV method")  
        
        elif key == ord('G'):
            image = reload_image(file)
            image = tograyown(image)
            print("'G' key pressed: convert to grey using own method")
        
        elif key == ord('c'):
            count= count + 1
            if count > 2:
                count = 0
            image = reload_image(file)
            print("'c' key pressed:")
            image = changecolor(image, count)

        elif key == ord('s'):
            image = reload_image(file)
            image = togray(image)
            cv2.imshow('image', image)
            print("'s' key pressed: Image to grayscale and smooth track bar created using openCV")
            cv2.createTrackbar('s', 'image', 0, 255, smooth)
        
        elif key == ord('S'):
            image = reload_image(file)
            image = tograyown(image)
            cv2.imshow('image', image)
            print("'S' key pressed: Image to grayscale and smooth track bar created using own method")
            cv2.createTrackbar('S', 'image', 0, 20, smoothown)
        
        elif key == ord('d'):
            image = reload_image(file)
            image = downsample(image)
            print("'d' key pressed: downsample 2 factor with no smoothing")
        
        elif key == ord('D'):
            image = reload_image(file)
            image = downsample_smooth(image)
            print("'D' key pressed: downsample 2 factor with smoothing")
        
        elif key == ord('x'):
            image = reload_image(file)
            image = togray(image)
            image = convdevnorm(image, 1)
            print("'x' key pressed: convert to grayscale, convolution with x derivative filter")

        elif key == ord('y'):
            image = reload_image(file)
            image = togray(image)
            image = convdevnorm(image, 0)
            print("'y' key pressed: convert to grayscale, convolution with x derivative filter")

        elif key == ord('m'):
            image = reload_image(file)
            image = togray(image)
            image = gradient_magnitude(image)
            print("'m' key pressed: show magnitude of gradient computed with x and y derivatives to range [0,255]")
        
        elif key == ord('p'):
            image = reload_image(file)
            image = togray(image)
            print("'p' key pressed: convert image to grayscale and plot gradient vectors")
            cv2.createTrackbar('p', 'image', 0,255, gradient_vectors)
        
        elif key == ord('r'):
            image = reload_image(file)
            image = togray(image)
            print("'r' key pressed: convert image to grayscale and rotate x degrees")
            cv2.createTrackbar('r', 'image', 0, 360, rotate)
            
        elif key == ord('h'):
            print("'h' key pressed: description of key functions:")
            help()

        elif key == 27:
            cv2.destroyAllWindows()
            print("'Esc' key pressed: Program Closed!")
            break
        else:
            print("Wrong key pressed, press 'h' for help")
        cv2.imshow('image', image)

In [20]:
main()

Write the file name path or press enter to capture an image: 

Press h for help
27
'Esc' key pressed: Program Closed!
